# Results of DDNet Trained on CIFAR10

In this notebook I display results in terms of accuracy on test data for DDNet models trained on the CIFAR10 dataset using various regularization techniques. The DDNet model and the CIFAR10 dataset were set up as in Hoffman 2019 (for all the models). The CIFAR10 data is preprocessed by normalizing using mean [0.5, 0.5, 0.5] and variance [0.5, 0.5, 0.5]. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Model parameters are initialized using Glorot initialization (See Glorot & Bengio 2010), expect for SVB regularization which uses orthogonal initialization. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization with a dropout rate of p_drop = 0.5. I also train a model with Jacobian Regularization without Dropout. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.1, and is reduced to 0.01 and 0.001 1/3 and 2/3s into training, respectively. The models are trained for 50 epochs.

### Imports and Model Loading

In [1]:
import jupyter_black
import numpy as np
import pandas as pd
import torch
from scipy import stats
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_CIFAR10
from model_classes import DDNet
from plotting_tools import get_random_img, generate_random_vectors
from tools import accuracy, compute_total_variation, ModelInfo

jupyter_black.load()

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_CIFAR10()

# Summary of model
summary_model = DDNet().to(device)
summary(summary_model, (3, 32, 32))

Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 30, 30]           1,792
            Conv2d-2           [-1, 64, 28, 28]          36,928
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 12, 12]          73,856
            Conv2d-5          [-1, 128, 10, 10]         147,584
         MaxPool2d-6            [-1, 128, 5, 5]               0
            Linear-7                  [-1, 256]         819,456
           Dropout-8                  [-1, 256]               0
            Linear-9                  [-1, 256]          65,792
          Dropout-10                  [-1, 256]               0
           Linear-11                   [-1, 10]           2,570
Total params: 1,147,978
Trainable params: 1,147,978
Non-trainable params: 0
---------------

In [5]:
# Load models
dataset = "cifar10"

model_names_set = [
    "model_no_reg",
    "model_l2",
    "model_jacobi",
    "model_jacobi_no_dropout",
    "model_svb",
]
model_names = []

for i in range(5):
    for name in model_names_set:
        model_names.append(f"{name}_{i}")

models = {name: ModelInfo(name, dataset) for name in model_names}

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [ ]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for name in model_names_set:
    accuracies = []
    for i in range(5):  # Assuming 5 versions of each model
        model_name = f"{name}_{i}"
        model = models[model_name].model
        acc = accuracy(model, test_loader, device)
        accuracies.append(acc)

    # Calculate mean accuracy
    mean_accuracy = np.mean(accuracies)

    # Calculate standard error
    std_error = stats.sem(accuracies)

    # Calculate confidence interval
    CI = std_error * stats.t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

    # Store results in the data dictionary
    accuracy_data[f"{name}"] = (mean_accuracy, CI)

In [ ]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(
    accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"]
)

# Convert the numbers from decimal to percentage and round to four decimal places
df = df.multiply(100).round(3)

# Create a new column that combines Mean Accuracy and Confidence Interval
df["Accuracy +/- CI (95%)"] = df.apply(
    lambda row: f"{row['Mean Accuracy']} +/- {row['Confidence Interval']}", axis=1
)

# Drop the original columns
df = df.drop(columns=["Mean Accuracy", "Confidence Interval"])

# Reset the index
df = df.reset_index().rename(columns={"index": "Model"})

display(df)

### Total Variation

Total variation is a measure for roughness/complexity in images. I generate 25 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the eight models (models with and without dropout) as rows, and the three different zoom levels as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level, for both isotropic and anisotropic total variation.

#### Isotropic Total Variation

In [ ]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Define the models
model_names = [
    "model_no_reg_0",
    "model_l2_0",
    "model_jacobi_0",
    "model_jacobi_no_dropout_0",
    "model_svb_0",
]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_isotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_isotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_isotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_isotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

In [ ]:
print(
    "Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_isotropic)

#### Anisotropic Total Variation

In [ ]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Define the models
model_names = [
    "model_no_reg_0",
    "model_l2_0",
    "model_jacobi_0",
    "model_jacobi_no_dropout_0",
    "model_svb_0",
]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_anisotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_anisotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_anisotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_anisotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

In [ ]:
print(
    "Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_anisotropic)